# LearnX: Personalized Learning Plans with Metaphor API

LearnX is a project that addresses a critical challenge in education: keeping pace with the ever-evolving world of knowledge. In an era where Large Language Models (LLMs) like ChatGPT are changing how we learn, the LearnX project leverages the Metaphor API to create a dynamic learning platform. This platform offers personalized learning/study plans, ensuring users stay current with the latest developments without the hassle of resource hunting. LearnX excels in providing structured learning plans, filling the gap where users often struggle to find a well-organized study path amidst the vast sea of information available on the internet. Harnessing the latest LLMs using LangChain and Metaphor API technologies, LearnX is the key to efficient and structured learning.

In [1]:
# Import necessary libraries
import re
import ast
from typing import List
from pprint import pprint
from bs4 import BeautifulSoup
from pydantic import BaseModel
from metaphor_python import Metaphor
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

## Helper Functions

### Using Metaphor API to create knowledge base

This part consists of four main functionalities:

**Step 1:** Getting the relevant links of the topic that the user chose from the internet using Metaphor API `search` functionality.

**Step 2:** Using the Metaphor API `get_contents` functionality to retrieve information from the links.

**Step 3:** Extracting pure text and removing HTML from the information gathered in Step 2.

**Step 4:** Creating the knowledge base by keeping only the 'extract' elements.


In [2]:
# Using the Metaphor API Search functionality to get links about a certain topic
def get_topic_links(topic,level):
    # Define  query 
    query = f"Latest articles on for learning {topic} for {level} level."
    # Send a search request to the Metaphor API
    search_response = metaphor.search(
        query=query,
        num_results=5,  #  can adjust the number of results as needed
        use_autoprompt = True
    )
    return search_response

# Using the Metaphor API get_contents functionality to get the information from the links
def get_content_for_topic(response):
    content_response_list = []
    if response.results:
        for result in response.results:
            # Get the HTML content for the URL
            content_response = metaphor.get_contents([result.id])
            content_response_list.append(content_response)
        print("Content retrieved and stored.")
    else:
        print("No content found for this topic.")
        print()  # Add an empty line for readability
    return content_response_list

# Function to extract text from HTML and update 'Extract' key
def extract_text_from_html(content_response_list):
    for item in content_response_list:
        try:
            # Access the 'extract' attribute within the 'DocumentContent' object
            html = item.contents[0].extract
            soup = BeautifulSoup(html, 'html.parser')
            item.contents[0].extract = soup.get_text()  # Update the 'extract' attribute
        except Exception as e:
            print(f"An error occurred: {str(e)}")
   
    return content_response_list

# Function to create one knowledge base from the gathered information
def create_knowledge_base(content_response_list):
    knowledge_base = ""
    for contents in content_response_list:
        knowledge_base += str(contents.contents[0].extract) # keep only the contents from the 'extract' key
    return knowledge_base

# Function to wrap everything together and get the final knowledge base
def get_knowledge_base(topic, level):
    # Use Metaphor API's Search to get latest article links
    response = get_topic_links(topic, level)
    # Get contents from the links by using Metaphor API's get_content()
    content_response_list = get_content_for_topic(response)
    # Extract text from HTML 
    content_response_list = extract_text_from_html(content_response_list)
    # Create a final knowledge base 
    knowledge_base = create_knowledge_base(content_response_list)
    return knowledge_base

### Get a structured study/learning plan from LLM


This part consists of two functionalities:

**Step 1:** Generating a personalized study plan with chapters and sub-topics using LLM. The LLM takes in the knowledge base, and user inputs like topic and experience level.

**Step 2:** Using the LLM to create a dictionary of the generated study plan.

In [3]:
# function to generate learning plan or study plan
def generate_learning_plan(topic, level, knowledge_base):
    #Create a template string for the prompt to be sent to LLM
    template_string = """
    Task: Generate a Structured Study/Learning Plan.

    Objective: Create a structured study/learning plan for a specified topic, taking into account the user's study level. This should be designed to assist users in planning their study approach efficiently.

    Instructions:
    Use the below user information to create the plan for the user. Assume the user has access to information in the form of web articles.
    1. {topic}: The topic for which the user needs a study plan.
    2. {level}: Whether the user is a beginner, amateur, or advanced in this subject matter.

    Output Response Template:
    1. Chapter 1: Introduction to [topic]
       - Subtopics or concepts to cover:
         - [Subtopic 1]
         - [Subtopic 2]
         - [Subtopic 3]
    2. Chapter 2:
       - Subtopics or concepts to cover:
         - [Subtopic 4]
         - [Subtopic 5]
    3. Chapter 3:
       - Subtopics or concepts to cover:
         - [Subtopic 6]
         - [Subtopic 7]
         
    Important Note: Continue this structure for the entire study plan, extending for further chapters. Please generate the structured learning plan directly without additional questions. Ensure that the output response includes a structured learning plan tailored to the user's study level. Use the provided output response template as a guideline for consistency.

    Feel free to use the provided topic information below to make a detailed and curated study/learning plan:
    Topic Information: ```{text}```
    """
    #Create prompt
    prompt_template = ChatPromptTemplate.from_template(template_string)
    prompt_messages = prompt_template.format_messages(
                        topic=topic,
                        level=level,
                        text=knowledge_base)
    # Call the LLM 
    llm_response = model(prompt_messages)
    return llm_response

#Function to create dictionary of the learning plan from the LLM's response. 
def create_dictionary_from_string(input_string):
    #prompt string
    template_string_for_json = """
    From the given string create a dictionary of chapters as key and the list of subtopics as value.
    STRING: {string}
    """
    #prompt template
    prompt_template_for_json = ChatPromptTemplate.from_template(template_string_for_json)
    prompt_messages_for_json = prompt_template_for_json.format_messages(string=input_string)
    # Call the LLM 
    llm_response = model(prompt_messages_for_json)
    return llm_response

#Function to extract dictionary from string.
def extract_dictionary_from_content(content):
    # Use regular expression to find the dictionary portion
    pattern = r'{[^{}]*}'
    # Find the matching dictionary using regex
    match = re.search(pattern, content)
    if match:
        # Extract the matched dictionary string
        dictionary_str = match.group(0)
        try:
            # Convert the dictionary string to a Python dictionary using `ast.literal_eval()`
            dictionary = ast.literal_eval(dictionary_str)
            return dictionary
        except Exception as e:
            print(f"Error converting dictionary string to Python dictionary: {e}")
            return None
    else:
        print("No dictionary found in the input string.")
        return None
    
# Function to wrap everything together and get the final learning plan in the form of a dictionary from the LLMs
def get_learning_plan(topic, level, knowledge_base):
    # get study plan as a string from the llm
    llm_response = generate_learning_plan(topic, level, knowledge_base)
    # create a dictionary of the plan. 
    llm_dict = create_dictionary_from_string(llm_response.content) #although the llm converts the plan in the form of a dictionary here, it is still a string.
    # convert the string into dictionary type
    final_dict = extract_dictionary_from_content(llm_dict.content)
    return final_dict

### Using Metaphor API to get resources and links for the study plan chapters and sub chapters 

Next, the dictionary of corresponding chapters and sub-topics created above is populated with links to resources from the web. Metaphor API's search method is used for this.

In [4]:
# Function to search for links related to a subtopic using Metaphor API
def search_links_for_subtopic(subtopic):
    # Make a search API call for the subtopic and get 3 links, keyword search
    search_result = metaphor.search(subtopic, num_results=3)  # You can adjust the number of results as needed
    # Extract and return the links as a list
    return [result.url for result in search_result.results]

# Function to populate the dictionary with links for each chapter and subtopic
def populate_links_dictionary(resulting_dictionary):
    new_resulting_dictionary = {}
    # Get three links for each sub-topic in each chapter
    for chapter, subtopics in resulting_dictionary.items():
        subtopic_links = {}
        for subtopic in subtopics:
            links = search_links_for_subtopic(subtopic)
            subtopic_links[subtopic] = links
        #append to the dictionary
        new_resulting_dictionary[chapter] = subtopic_links
    return new_resulting_dictionary

### Custom quiz generator helper functions

This is the final phase of the project. For this phase, an LLM is used to generate a quiz based on the knowledge base created.

In [5]:
# Function to generate the quiz
def generate_quiz(topic, final_dict, knowledge_base):
    template_string_for_quiz = """Generate a quiz on {topic} with 10-15 multiple choice questions (3-4 answer options). You can use the following topic information and study/learning plan:
    topic information:
    {knowledge_base}

    Study/Learning Plan:
    {learning_plan}

    Important Note: The quiz should roughly evaluate the study/learning plan. Also give the correct answer key for the quiz at the end.
    """
    #Create a prompt template for the LLM
    prompt_template_for_quiz = ChatPromptTemplate.from_template(template_string_for_quiz)
    prompt_messages_for_quiz = prompt_template_for_quiz.format_messages(
                    topic = topic,
                    learning_plan = final_dict,
                    knowledge_base = knowledge_base)
    quiz_response = model(prompt_messages_for_quiz)
    return quiz_response

### Display helper functions

These are helper functions to display the dictionaries formed in a neat and clean manner.

In [6]:
# Function to display the list of links extracted using Metaphor API for the knowledge base
def display_list_resource(response):
    if response.results:
        for result in response.results:
            print(f"Title: {result.title}")
            print(f"URL: {result.url}")
            print()  # Add an empty line for readability
    else:
        print("No relevant articles found found.")
        
# Function to print the final resulting dictionary
def print_resulting_dictionary(resulting_dictionary):
    for chapter, subtopics in resulting_dictionary.items():
        print(f"Chapter: {chapter}")
        for subtopic, links in subtopics.items():
            print(f"    Subtopic: {subtopic}")
            for idx, link in enumerate(links, start=1):
                print(f"        Link {idx}: {link}")
        print()
        print()

    return        

## Main Functionality 

### Get User Input

In [7]:
# User input
topic = "Amazon Simple Queue Service" # The topic the user wants to learn
level = "beginner" # Beginner/Intermediate/Advanced

In [8]:
# ### Set API Key
api_key = "YOUR-METAPHOR-API-KEY"
metaphor = Metaphor(api_key)

## OpenAI API
openai_api_key = "YOUR-OPENAI-API-KEY"
model = ChatOpenAI(openai_api_key=openai_api_key)

### Creating the knowledge base using Metaphor API functions

In [9]:
knowledge_base = get_knowledge_base(topic, level)

Content retrieved and stored.


### Using the LLM to generate a personalized learning plan/study plan

In [10]:
final_dict = get_learning_plan(topic, level, knowledge_base)

### Using Metaphor API to get links and resources for the topics in the generated learning path

In [11]:
new_resulting_dictionary = populate_links_dictionary(final_dict)
print_resulting_dictionary(new_resulting_dictionary)

Chapter: Chapter 1
    Subtopic: Overview of Amazon SQS
        Link 1: https://docs.aws.amazon.com/AWSSimpleQueueService/latest/SQSDeveloperGuide/welcome.html
        Link 2: https://aws.amazon.com/pub-sub-messaging/
        Link 3: https://aws.amazon.com/sqs/getting-started/
    Subtopic: Benefits and use cases of Amazon SQS
        Link 1: https://aws.amazon.com/sqs/
        Link 2: https://aws.amazon.com/message-queue/
        Link 3: https://aws.amazon.com/pt/sqs/
    Subtopic: Comparison with other AWS messaging services (Amazon SNS, Amazon MQ, Amazon Kinesis Streams)
        Link 1: https://bitesizedserverless.com/bite/serverless-messaging-latency-compared
        Link 2: https://www.amazonaws.cn/en/sqs/
        Link 3: https://docs.aws.amazon.com/sns/latest/dg/sns-event-sources.html
    Subtopic: Cost model of AWS SQS
        Link 1: https://aws.amazon.com/sqs/pricing/?nc1=h_ls
        Link 2: https://aws.amazon.com/sns/pricing/
        Link 3: https://aws.amazon.com/sqs/


Cha

### Generate a customized quiz on the topic as final assesment

In [12]:
llm_quiz_response = generate_quiz(topic, final_dict, knowledge_base)
print(llm_quiz_response.content)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


1. What is the purpose of Amazon Simple Queue Service (SQS)?
a) To send time-critical messages to multiple subscribers
b) To store and move data between distributed application components
c) To process streaming big data in real-time
d) To create scalable and reliable EC2 applications

2. How is Amazon SQS different from Amazon SNS?
a) Amazon SQS allows for time-critical messaging, while Amazon SNS enables asynchronous messaging.
b) Amazon SQS is a message queue service, while Amazon SNS is a message notification service.
c) Amazon SQS supports industry-standard APIs and protocols, while Amazon SNS does not.
d) Amazon SQS guarantees at-least-once delivery, while Amazon SNS guarantees exactly-once processing.

3. Which type of queue should be used if you need absolute guarantees on message order and processing?
a) Standard queue
b) FIFO queue
c) Delay queue
d) Priority queue

4. What is the cost model of AWS SQS?
a) Pay-as-you-go pricing based on message size and number of requests
b) F